In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\hp\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\hp\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [5]:
data=pd.read_csv("C:/Users/hp/Downloads/tripadvisor_hotel_review/tripadvisor_hotel_reviews.csv")

In [6]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


# data cleaning

In [7]:
data.shape

(20491, 2)

In [8]:
import re
import nltk
from nltk.corpus import stopwords

In [9]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
def clean_text(text):
    text=re.sub(r'\W',' ', text)
    test=re.sub(r'\s+',' ', text)
    text=text.lower()
    text=text.strip()
    return text

In [11]:
 ### Apply the cleaning function to the review column
data['cleaned_text']=data['Review'].apply(clean_text)

# Tokenization and Stopword Removal

In [12]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Tokenization and stopword removal function
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    return tokens 

In [14]:
data['tokens'] = data['cleaned_text'].apply(tokenize_and_remove_stopwords)

In [15]:
print(data['tokens'])

0        [nice, hotel, expensive, parking, got, good, d...
1        [ok, nothing, special, charge, diamond, member...
2        [nice, rooms, 4, experience, hotel, monaco, se...
3        [unique, great, stay, wonderful, time, hotel, ...
4        [great, stay, great, stay, went, seahawk, game...
                               ...                        
20486    [best, kept, secret, 3rd, time, staying, charm...
20487    [great, location, price, view, hotel, great, q...
20488    [ok, looks, nice, modern, outside, desk, staff...
20489    [hotel, theft, ruined, vacation, hotel, opened...
20490    [people, talking, ca, n, believe, excellent, r...
Name: tokens, Length: 20491, dtype: object


# Convert Text to Numerical Data

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:

# Combine tokens back into strings for vectorization
data['final_review'] = data['tokens'].apply(lambda x: ' '.join(x))

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the reviews
X = vectorizer.fit_transform(data['final_review']).toarray()

# Prepare Target Variable

In [18]:
# Example: Assuming ratings from 1 to 5, where 1-2 are negative, 3 is neutral, and 4-5 are positive
def get_sentiment(rating):
    if rating <= 2:
        return 'negative'
    elif rating == 3:
        return 'neutral'
    else:
        return 'positive'


In [19]:
data['sentiment'] = data['Rating'].apply(get_sentiment)
y = data['sentiment']

# Train-Test Split

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

# Initialize the classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train, y_train)

MultinomialNB()

# Model Evaluation

In [22]:
from sklearn.metrics import classification_report, accuracy_score

# Make predictions
y_pred = classifier.predict(X_test)

# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.8241034398633813
Classification Report:
              precision    recall  f1-score   support

    negative       0.85      0.57      0.68       625
     neutral       0.50      0.00      0.00       432
    positive       0.82      0.99      0.90      3042

    accuracy                           0.82      4099
   macro avg       0.72      0.52      0.53      4099
weighted avg       0.79      0.82      0.77      4099

